Here we are looking to create a dashboard for all antenna autocorrelations.

In [14]:
import numpy as np
import pandas as pd
import hvplot.pandas
import hvplot.streamz
import holoviews as hv
import matplotlib.pyplot as plt
from cosmic.hashpipe_aux import redis_obj
from cosmic.fengines import ant_remotefeng_map
import operator as op
import numpy as np
import panel as pn
from streamz.dataframe import PeriodicDataFrame
pn.extension()

In [15]:
#fetch the ant2feng dict
ant_feng_dict = ant_remotefeng_map.get_antennaFengineDict(redis_obj)
antnames = list(ant_feng_dict.keys())

NUM_CHANNELS = 1024
NUM_TUNINGS = 4


In [18]:
def ant_dataFrame():
    #fetch the ant2feng dict
    ant_feng_dict = ant_remotefeng_map.get_antennaFengineDict(redis_obj)
    feng_dict = {}
    index = 0
    for antname, feng in ant_feng_dict.items():
        autocorr = np.array(feng.autocorr.get_new_spectra(),dtype=np.float64)
        n_ifs, n_chans = autocorr.shape
        for i in range(n_ifs):
            spectra_dat = 10*np.log10(autocorr[i])
            for c in range(n_chans):
                feng_dict[index] = {
                    "SPECTRA" : spectra_dat[c],
                    "ANT": f"""{antname}""",
                    "CHANNEL" : c,
                    "IF" : i
                }
                index += 1
    df = pd.DataFrame(feng_dict).transpose()
    return df

In [19]:
v=ant_dataFrame()
v.head()

,SPECTRA,ANT,CHANNEL,IF
0,-67.9622,ea14,0,0
1,-66.2379,ea14,1,0
2,-64.5516,ea14,2,0
3,-65.202,ea14,3,0
4,-65.1336,ea14,4,0


In [20]:
def streaming_ant_dataFrame(**kwargs):
    return ant_dataFrame()

In [21]:
df = PeriodicDataFrame(streaming_ant_dataFrame, interval = '10s')

How do we want to process the above pipeline? We want to expose the following:
- Number of channels
- Phase
- Power

In [22]:
# antnames = pn.widgets.CheckBoxGroup(
# name = 'Antenna', 
# options=list(ant_feng_dict.keys()),
# inline=True
# )
# channels = pn.widgets.DiscreteSlider(
# name = 'Number of Channels',
# options = list(np.arange(NUM_CHANNELS)+1),
# value = NUM_CHANNELS
# )


In [23]:
def topPanelMaker():    
    pn_realtime = pn.Column("# Autocorrelation Dashboard")
    for ant in antnames:
        pn_realtime.append(pn.Column(f"""## Antenna {ant}"""))
        pn_realtime.append(pn.Column(
        pn.Row(
            df.hvplot.line(x="CHANNEL",y="SPECTRA",title="Spectral Power [dB]"),
            )
        ))
    return pn_realtime

In [24]:
# widgets = pn.Column("# Autocorrelation Dashboard", antnames,channels)
plots = pn.Row(topPanelMaker)#,widgets)
plots.servable()

Row
    [0] Column
        [0] Column()
        [1] Row
            [0] Column
                [0] Markdown(str)
                [1] Column
                    [0] Markdown(str)
                [2] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [3] Column
                    [0] Markdown(str)
                [4] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [5] Column
                    [0] Markdown(str)
                [6] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [7] Column
                    [0] Markdown(str)
                [8] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [9] Column
                    [0] Markdown(str)
                [10] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [11] Column
                    [0] Markdown(str)
                [12] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [13] Column
                    [0] Markdown(str)
                [14] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [15] Column
                    [0] Markdown(str)
                [16] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [17] Column
                    [0] Markdown(str)
                [18] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [19] Column
                    [0] Markdown(str)
                [20] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [21] Column
                    [0] Markdown(str)
                [22] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [23] Column
                    [0] Markdown(str)
                [24] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [25] Column
                    [0] Markdown(str)
                [26] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [27] Column
                    [0] Markdown(str)
                [28] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [29] Column
                    [0] Markdown(str)
                [30] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [31] Column
                    [0] Markdown(str)
                [32] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [33] Column
                    [0] Markdown(str)
                [34] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [35] Column
                    [0] Markdown(str)
                [36] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [37] Column
                    [0] Markdown(str)
                [38] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)
                [39] Column
                    [0] Markdown(str)
                [40] Column
                    [0] Row
                        [0] HoloViews(DynamicMap)

In [10]:
# pn_realtime = pn.Column(f"""## Antenna {"ea14"}""")
# pn_realtime.append(pn.Column(
#         pn.Row(
#             df[
#                (df.IF==1)  & (df.ANT=="ea14")
#             ].MAG.hvplot.line(x="CHANNEL",y="MAG",title="MAG"),
#             df[
#                 (df.IF==1) & (df.ANT=="ea14")
#             ].hvplot.line(x="CHANNEL",y="PHASE",title="PHASE")
#             )))
# pn_realtime.servable()